# Extração de Entidades NER e Parâmetros Clínicos com LLMs

Este script foi concebido para realizar o Reconhecimento de Entidades Nomeadas (NER) em diários clínicos de cardiologia. Utilizando o poder de um modelo de linguagem avançado (LLM), especificamente o google/gemma-3-12b-it, o script processa texto não estruturado para identificar, extrair e classificar informações cruciais. As entidades médicas, como sintomas, diagnósticos, medicamentos e sinais vitais, são extraídas e estruturadas num formato JSON claro e válido. O sistema é flexível, permitindo a execução tanto em CPU, através da biblioteca llama-cpp-python, como em GPUs potentes, com o auxílio da biblioteca transformers da Hugging Face, tornando a análise de dados clínicos mais eficiente e automatizada.

###  Importações e Configurações Globais
Este bloco de código inicial é responsável por configurar o ambiente de execução do script. Primeiramente, importa as bibliotecas essenciais como os, re e sys para manipulação do sistema e expressões regulares. Em seguida, define um interruptor crucial, USE_LLAMA_CPP, que permite ao utilizador escolher entre dois backends de inferência: llama-cpp-python para uma execução otimizada em CPU, ou a biblioteca transformers da Hugging Face, que é mais adequada para ambientes com GPUs potentes. O código também realiza importações condicionais com base nesta escolha e estabelece parâmetros globais que irão governar o comportamento do modelo, como o tamanho máximo do contexto (N_CTX), o número de threads (N_THREADS) e os parâmetros de geração de texto (MAX_TOKENS, TEMPERATURE, etc.).

In [ ]:
import gc
import os
import re
import sys
import textwrap

# --- Interruptor de Backend ---
# Altere esta variável para escolher o motor de inferência.
# True: Usa llama-cpp-python (otimizado para CPU, mais rápido sem GPU).
# False: Usa a biblioteca transformers da Hugging Face (requer GPU potente para este modelo).
USE_LLAMA_CPP = True

# --- Importações Condicionais ---
if USE_LLAMA_CPP:
    from llama_cpp import Llama
else:
    try:
        import torch
        from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
    except ImportError:
        print("Erro: A biblioteca 'transformers' e/ou 'torch' não estão instaladas.")
        print("Para usar o backend da Hugging Face, instale-as com: pip install transformers torch")
        sys.exit(1)

# --- Configurações Globais ---
# Parâmetros para a execução do modelo e processamento de texto.
N_CTX = 131072
N_THREADS = 16  # Relevante principalmente para llama-cpp
MAX_TOKENS = 4096
TEMPERATURE = 1
TOP_P = 0.95
TOP_K = 60
MIN_P = 0.01

### Inicialização do Modelo (Modularizada)
A função initialize_model encapsula a lógica de carregamento do modelo de linguagem. Dependendo do valor da variável USE_LLAMA_CPP, esta função segue um de dois caminhos. Se USE_LLAMA_CPP for verdadeiro, ela descarrega e inicializa o modelo gemma-3-12b-it numa versão otimizada para CPU (formato GGUF), configurando-o para forçar a computação em CPU (n_gpu_layers=0). Caso contrário, o script tenta carregar o modelo completo através da biblioteca transformers, que requer PyTorch e uma GPU com memória de vídeo substancial (>24GB). A função inclui gestão de erros robusta para lidar com falhas durante o processo de inicialização e informa o utilizador sobre qual backend está a ser utilizado

In [2]:
# --- Inicialização do Modelo (Modularizada) ---
def initialize_model():
    """
    Inicializa e carrega o modelo e o tokenizer com base no backend selecionado.
    """
    print(f"A inicializar o modelo com o backend: {'llama-cpp-python' if USE_LLAMA_CPP else 'Hugging Face Transformers'}...")

    if USE_LLAMA_CPP:
        try:
            model = Llama.from_pretrained(
                repo_id="google/gemma-3-12b-it-qat-q4_0-gguf",
                filename="gemma-3-12b-it-q4_0.gguf",
                n_ctx=N_CTX,
                n_threads=N_THREADS,
                n_gpu_layers=0,  # Forçar CPU
                verbose=False
            )
            tokenizer = None  # llama-cpp lida com a tokenização internamente
            print("Modelo Llama.cpp inicializado com sucesso.")
            return model, tokenizer
        except Exception as e:
            print(f"Erro ao inicializar o modelo Llama.cpp: {e}")
            sys.exit(1)
    else:
        # --- AVISO: Requer GPU com >24GB de VRAM para o modelo 12B ---
        if not torch.cuda.is_available():
            print("AVISO: Nenhuma GPU detetada. O backend 'transformers' será extremamente lento em CPU.")

        try:
            model_name = "google/gemma-3-12b-it"
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto",  # Tenta usar GPU se disponível
                torch_dtype=torch.bfloat16, # Otimização para GPUs mais recentes
                attn_implementation="flash_attention_2" # Requer hardware compatível
            )
            print("Modelo Hugging Face Transformers inicializado com sucesso.")
            return model, tokenizer
        except Exception as e:
            print(f"Erro ao inicializar o modelo Hugging Face: {e}")
            sys.exit(1)

### Função de Geração de NER
A função generate_ner_json_streaming é o núcleo da lógica de análise. Ela recebe o texto de um diário clínico e constrói um prompt detalhado para o modelo de linguagem. Este prompt instrui o modelo a agir como um especialista em NER médico e a extrair entidades específicas (Sintomas, Diagnósticos, Medicamentos, etc.). As instruções são muito precisas, definindo um formato de saída JSON estrito, incluindo as chaves esperadas (id, classe, valor, etc.) e as regras para o seu preenchimento. A função envia o prompt para o modelo carregado e processa a resposta em streaming, imprimindo o JSON gerado em tempo real. Esta abordagem é eficiente e fornece feedback imediato ao utilizador.



In [ ]:
# --- Função de Geração de NER ---
def generate_ner_json_streaming(text: str, model, tokenizer):
    """
    Gera entidades NER utilizando o backend configurado.

    Args:
        text (str): O texto do diário médico a ser analisado.
        model: A instância do modelo carregado.
        tokenizer: O tokenizer correspondente (ou None para llama-cpp).
    """
    instructions = textwrap.dedent("""
    És um assistente especializado em Reconhecimento de Entidades Nomeadas (NER) na área médica.
    O teu objetivo é identificar e extrair termos relevantes de um diário médico de cardiologia.
    Formata a saída como uma lista JSON válida, onde cada objeto representa uma entidade identificada.

    Instruções para o formato JSON:
    1. A saída DEVE ser uma lista JSON `[...]`.
    2. Cada objeto na lista deve ter as seguintes chaves:
        - `id`: (Inteiro) ID numérico sequencial da entidade (1, 2, 3...).
        - `classe`: (String) A classe da entidade (use os códigos: S, D, M, DS, PM, SV, R, P).
        - `valor`: (String) O texto exato da entidade identificada.
        - `estado_clinico`: (String) No caso de um diagnóstico ou sintoma, indica se a pessoa tem essa condição ("+"), se não tem ("-") ou se é uma possibilidade ("?"). Omita se não aplicável.
        - `loc-temp`: (String) O valor de localização temporal deve estar presente apenas caso não seja o momento atual (hoje) (ex: "ontem", "2014", "na infância", "há 3 dias"). Omita se não aplicável.
        - `quem`: (String) O valor de quem tem uma condição, diagnóstico, medicação, etc., presente apenas no caso de não se referir ao próprio paciente (ex: "familiar(filho, mãe, etc.)"). Omita se não aplicável.
        - `relacionamento_id`: (Inteiro) Inclua esta chave *apenas* quando 2 classes estão relacionadas, indicando o `id` da entidade relacionada (ex: o `id` do Medicamento para uma Dosagem). Omita esta chave noutros casos.
    3. Certifique-se de que a saída final seja um JSON estritamente válido. Não inclua nenhum texto antes ou depois da lista JSON, nem marcadores como ```json.

    Classes permitidas e seus significados:
        - S: Sintoma, D: Diagnóstico, M: Medicamento, DS: Dosagem, PM: Procedimento Médico, SV: Sinal Vital, R: Resultado, P: Progresso

    Exemplo de Saída JSON Válida:
    [
      {"id": 1, "classe": "S", "valor": "Dor torácica", "estado_clinico": "+", "loc-temp": "2015", "quem": "mãe"}
    ]
    """).strip()

    user_input = textwrap.dedent(f"Texto para Análise:\n{text}").strip()
    prompt = f"{instructions}\\n\\n{user_input}\\n\\nLista JSON de Entidades Identificadas:\\n"

    print("\\n--- Saída do Modelo (Streaming) ---")
    try:
        if USE_LLAMA_CPP:
            response_stream = model.create_completion(
                prompt=prompt, max_tokens=MAX_TOKENS, temperature=TEMPERATURE,
                top_p=TOP_P, top_k=TOP_K, stream=True, min_p=MIN_P,
            )
            for chunk in response_stream:
                if delta := chunk['choices'][0].get('text'):
                    print(delta, end="", flush=True)
        else:
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
            _ = model.generate(**inputs, max_new_tokens=MAX_TOKENS, temperature=TEMPERATURE,
                               top_p=TOP_P, top_k=TOP_K, streamer=streamer)

    except Exception as e:
        print(f"\\n[Erro na geração] {str(e)}")
    finally:
        print("\\n--- Fim da Saída do Modelo ---")

### Função de Carregamento e Processamento de Ficheiros
A função load_and_process_file gere a leitura do ficheiro de entrada que contém os diários clínicos. Ela verifica se o ficheiro existe e, em seguida, lê-o linha por linha. Utilizando uma expressão regular (re.compile), a função consegue identificar o início de cada nova entrada no diário (delimitada por um número seguido de um hífen). Ao detetar uma nova entrada, processa a secção anterior acumulada, enviando-a para a função generate_ner_json_streaming para extração das entidades. Este método permite processar ficheiros com múltiplos registos de forma sequencial e organizada.

In [4]:
# --- Função de Carregamento e Processamento de Ficheiros ---
def load_and_process_file(file_name: str, model, tokenizer):
    if not os.path.exists(file_name):
        print(f"Erro: Ficheiro de entrada '{file_name}' não encontrado.")
        return

    print(f"\\nProcessando o ficheiro: {file_name}")
    with open(file_name, 'r', encoding='utf-8') as file:
        entry_index = 0
        current_section = ""
        entry_header_pattern = re.compile(r"^(\\d+)\\s*-")

        for line in file:
            match = entry_header_pattern.match(line)
            if match:
                if current_section.strip():
                    print(f"\\n\\n=== Processando Entrada {entry_index} ===")
                    generate_ner_json_streaming(current_section.strip(), model, tokenizer)
                current_section = line
                entry_index = int(match.group(1))
            else:
                current_section += line

        if current_section.strip():
            print(f"\\n\\n=== Processando Entrada {entry_index} ===")
            generate_ner_json_streaming(current_section.strip(), model, tokenizer)

    print("\\n\\n=== Processamento Concluído ===")

### Bloco de Execução Principal
Este é o ponto de entrada do script. A sua execução está protegida pela condição if __name__ == "__main__", uma prática recomendada em Python. O bloco orquestra todo o fluxo de trabalho: primeiro, chama a função initialize_model para carregar o modelo e o tokenizer. Se o modelo for carregado com sucesso, define o nome do ficheiro de entrada (diarios-cardiologia-amostra.txt) e invoca a função load_and_process_file para iniciar a análise. Por fim, após o processamento de todos os ficheiros, executa uma coleta de lixo (gc.collect()) para libertar memória, uma etapa especialmente importante ao lidar com modelos de grande dimensão.

In [5]:
# --- Bloco de Execução Principal ---
if __name__ == "__main__":
    model, tokenizer = initialize_model()

    if model:
        INPUT_FILE = "diarios-cardiologia-amostra.txt"
        load_and_process_file(INPUT_FILE, model, tokenizer)

    print("\\nExecutando a coleta de lixo final...")
    gc.collect()
    if not USE_LLAMA_CPP and torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Script finalizado.")

A inicializar o modelo com o backend: llama-cpp-python...


c:\Users\joaop\OneDrive\Ambiente de Trabalho\Prologica\Prologica_new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


Modelo Llama.cpp inicializado com sucesso.
\nProcessando o ficheiro: diarios-cardiologia-amostra.txt
\n\n=== Processando Entrada 0 ===
\n--- Saída do Modelo (Streaming) ---
```json
[
  {"id": 1, "classe": "S", "valor": "Dor torácica"},
  {"id": 2, "classe": "SV", "valor": "FC- 57 bpm"},
  {"id": 3, "classe": "SV", "valor": "FC- 72 bpm"},
  {"id": 4, "classe": "SV", "valor": "FC- 61 bpm"},
  {"id": 5, "classe": "SV", "valor": "TA 133/72"},
  {"id": 6, "classe": "D", "valor": "BAV 1º"},
  {"id": 7, "classe": "S", "valor": "cansaço"},
  {"id": 8, "classe": "D", "valor": "Miocardiopatia dilatada"},
  {"id": 9, "classe": "SV", "valor": "VE-47 mm"},
  {"id": 10, "classe": "SV", "valor": "SIV-7mm"},
  {"id": 11, "classe": "SV", "valor": "PP-6 mm"},
  {"id": 12, "classe": "SV", "valor": "Fs 34%"},
  {"id": 13, "classe": "SV", "valor": "TAPSE- 24mm"},
  {"id": 14, "classe": "D", "valor": "IC"},
  {"id": 15, "classe": "D", "valor": "MCD"},
  {"id": 16, "classe": "M", "valor": "dapa"},
  {"id": 1